In [ ]:
import os
import numpy as np
# import pandas as pd
import xarray as xr

from esdglider import gcp, glider, utils

deployment_name = "amlr03-20231128"
# deployment_name = "amlr08-20220513"
# deployment_name = "unit_1024-20250224"

mode = "delayed"


# Standard 
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket("amlr-gliders-deployments-dev", deployments_path, ro=False)
deployment_info = {
    "deploymentyaml": os.path.join(config_path, f"{deployment_name}.yml"), 
    "mode": mode, 
}
paths = glider.get_path_deployment(deployment_info, deployments_path)

dir_ts = paths["tsdir"]
path_raw = os.path.join(dir_ts, f"{deployment_name}-{mode}-raw.nc")
path_sci = os.path.join(dir_ts, f"{deployment_name}-{mode}-sci.nc")
path_eng = os.path.join(dir_ts, f"{deployment_name}-{mode}-eng.nc")

In [ ]:
ds_raw = xr.load_dataset(paths["tsrawpath"])
df_raw = ds_raw.to_pandas()
display(ds_raw)

In [ ]:
ds_eng = xr.load_dataset(paths["tsengpath"])
df_eng = ds_eng.to_pandas()
display(ds_eng)

ds_sci = xr.load_dataset(paths["tsscipath"])
df_sci = ds_sci.to_pandas()
display(ds_sci)

In [ ]:
ds = ds_sci
dt = "2025-02-25"
ds_sub = ds.sel(time=slice(f"{dt} 17:00", dt))
ds_sub = utils.data_var_reorder(ds_sub, ["profile_index", "profile_direction"])
ds_sub

## Profile checks

In [ ]:
display(utils.check_profiles(ds_eng))
display(utils.check_profiles(ds_sci))

# idx_list = [454.5, 578.5, 582.5, 586.5, 602.5, 626.5, 628.5, 630.5, 632.5, 634.5, 636.5, 650.5, 652.5]
# p1 = utils.check_profiles(ds_sci)
# p1e = p1[np.isin(p1["profile_ibndex"], idx_list)]
# p1e
df_check = utils.check_profiles(ds_raw)
display(df_check)

In [ ]:
# ds = ds_raw
# dt = "2022-05-14"
# ds_sub = ds.sel(time=slice(f"{dt} 18:00", dt))
# ds_sub

<!-- ## Testing new profile parameters -->

## Exploring new profile calculations

In [ ]:
ds_touse = ds_raw

# import math
vars_keep = ["longitude", "latitude", "depth", #"depth_ctd", 
                "profile_index", "profile_direction"]
x_orig = utils.data_var_reorder(ds_touse, vars_keep)
regions_df_orig = utils.calc_regions(x_orig)
# print(x_orig.profile_index.attrs)

# pgutils.get_profiles_new(x, filt_time=20)
p1, p2 = utils.findProfiles(
        x_orig.time.values,
        x_orig.depth.values,
        stall=3,
        shake=20, 
        interrupt = 180,
        inversion = 3, 
        length=10, 
        period=0, 
        # stall=10,
        # interrupt=600,
    )

orig = ["profile_index_orig", "profile_direction_orig"]
x = x_orig.rename_vars({"profile_index": orig[0], "profile_direction": orig[1]})
x["profile_index"] = ('time', p1, {})
x["profile_direction"] = ('time', p2, {})
x = utils.data_var_reorder(x, (vars_keep + orig))

# x["dives"] = ('time', gt.utils.calc_dive_number(x.time.values, x.depth.values, dive_depth_threshold=15), {})
# display(x)

ds = x
# dt = "2025-02-24"
dt = "2025-02-24"
ds_sub = ds.sel(time=slice(f"{dt} 23:00", "2025-02-25"))
ds_sub

In [ ]:
# Check profiles
prof_summ = utils.check_profiles(x)
print(f"Dives: {np.count_nonzero(prof_summ.profile_description.values == "Dive")}")
print(f"Climbs: {np.count_nonzero(prof_summ.profile_description.values == "Climb")}")
prof_summ

In [ ]:
# Check profiles on eng and sci datasets
x_eng = utils.join_profiles(ds_eng, utils.calc_profile_summary(x))
display(utils.check_profiles(x_eng))

x_sci = utils.join_profiles(ds_sci, utils.calc_profile_summary(x))
display(utils.check_profiles(x_sci))

In [ ]:
x_sci

In [ ]:
# regions_df_orig = utils.calc_regions(x_orig)
# regions_df = utils.calc_regions(x)
# # display(regions_df)
# # print(np.argwhere(regions_df_orig.profile_description == np.roll(regions_df_orig.profile_description, 1)))

# # print(regions_df["profile_direction"].value_counts())
# print("------------")
# print(f"Climbs orig: {np.count_nonzero(regions_df_orig["profile_direction"] == -1)}")
# print(f"Dives orig: {np.count_nonzero(regions_df_orig["profile_direction"] == 1)}")
# utils.check_profiles(x_orig)

# print("------------")
# print(f"Climbs: {np.count_nonzero(regions_df["profile_direction"] == -1)}")
# print(f"Dives: {np.count_nonzero(regions_df["profile_direction"] == 1)}")
# utils.check_profiles(x)

# regions_df
# regions_df_orig
# 
# 
# prof_summ_df_orig = utils.check_profiles(x_orig)
# prof_summ_df = utils.check_profiles(x)

# between_df_orig = prof_summ_df_orig[prof_summ_df_orig["profile_index"] % 1 == 0.5]
# between_df = prof_summ_df[prof_summ_df["profile_index"] % 1 == 0.5]

# # print((between_df_orig.end_time - between_df_orig.start_time).describe())
# # print((between_df.end_time - between_df.start_time).describe())

# print(abs(between_df_orig.end_depth - between_df_orig.start_depth).describe())
# print(abs(between_df.end_depth - between_df.start_depth).describe())

# prof_summ_df_orig

In [ ]:
z = xr.load_dataset("/home/sam_woodman_noaa_gov/depfrbyrd_amlr03_sgg3_L1_2023-11-27_data_rt.nc")
z

In [ ]:
# pgutils.get_profiles_new(x, filt_time=20)
p1, p2 = utils.findProfiles(
        z.time.values,
        z.depth.values,
        stall=20,
        # shake=200, 
        # interrupt = 600,
        # inversion = 10
    )

# orig = ["profile_index_orig", "profile_direction_orig"]
# x = x_orig.rename_vars({"profile_index": orig[0], "profile_direction": orig[1]})
z["profile_index_new"] = ('time', p1, {})
z["profile_direction_new"] = ('time', p2, {})
# x = utils.data_var_reorder(x, (vars_keep + orig))
z

In [ ]:
import matplotlib.pyplot as plt
depth = np.array([4, 4, 3, 2, 3, 4, 4, 5, 6, 6, 6, 5, 4, 4, 5, 3, 2, 3, 1, 1, 0, 4, 4])
x = np.arange(0, len(depth))

profile_index, profile_direction = utils.findProfiles(x, depth)
# %    figure
# %    subplot(3, 1, 1, 'XGrid','on','YGrid','on', 'NextPlot', 'add')
# %    stairs(profile_direction, '-g')
# %    subplot(3, 1, 2, 'XGrid','on','YGrid','on', 'NextPlot', 'add')
# %    plot(depth, '-db')
# %    subplot(3, 1, 3, 'XGrid','on','YGrid','on', 'NextPlot', 'add')
# %    stairs(profile_index, '-r')
# %    [profile_index, profile_direction] = findProfiles(depth, 'stall', 1.5)
# %    stairs(profile_index, '-c')
# %    [profile_index, profile_direction] = ...
# %       findProfiles(depth, 'stall', 1.5, 'inversion', 1.5, 'interrupt', inf)
# %    stairs(profile_index, '-m')

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
axs[0].grid(True)
axs[0].stairs(profile_direction, color='g')
axs[0].set_title('Profile Direction')

axs[1].grid(True)
axs[1].plot(depth, '-db')
axs[1].set_title('Depth')
axs[1].invert_yaxis()

axs[2].grid(True)
axs[2].stairs(profile_index, color='r')
axs[2].set_title('Profile Index')

# Second call with stall parameter
profile_index, profile_direction = utils.findProfiles(x, depth, stall=1.5)
axs[2].stairs(profile_index, color='c', label='stall=1.5')

# Third call with more parameters
profile_index, profile_direction = utils.findProfiles(
    x, depth, stall=1.5, inversion=1.5, interrupt=np.inf)
axs[2].stairs(profile_index, color='m', label='stall=1.5, inversion=1.5, interrupt=inf')

axs[2].legend()
plt.tight_layout()
plt.show()